In [ ]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import datetime
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/VehiclePositions_A.pb')
feed.ParseFromString(response.content)


entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))
# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'vehicle_id': vehicle.vehicle.id,
            'trip_id': vehicle.trip.trip_id,
            'route_id': vehicle.trip.route_id,
            'latitude': vehicle.position.latitude,
            'longitude': vehicle.position.longitude,
            'bearing': vehicle.position.bearing,
            'speed': vehicle.position.speed,
            'timestamp': datetime.datetime.fromtimestamp(vehicle.timestamp),
            'stop_id':vehicle.stop_id,
            'current_stop_sequence':vehicle.current_stop_sequence,
            'status': vehicle.vehicle_stop_status,
            'current_date': datetime.date.today(),
            'day': datetime.date.today().isoweekday()
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

In [6]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
import requests
import datetime
import os

ref_df=pd.read_csv('../data/rozklad/stop_times.txt',delimiter=',')
positions_df=pd.read_csv('../data/positions/vehicle_positions_2025-04-30_004744.csv')

merged = pd.merge(positions_df,ref_df[['trip_id', 'stop_id', 'departure_time']],on=['trip_id', 'stop_id'],how='left')

merged.head()

,vehicle_id,trip_id,route_id,latitude,longitude,speed,timestamp,stop_id,current_stop_sequence,current_status,current_date,day,departure_time
0,320,block_542_trip_11_service_1,NaN,50.174305,19.913160,0.0,2025-04-30 00:47:29,stop_1313_296801,1,1,2025-04-30,3,24:46:00
1,508,block_509_trip_10_service_1,NaN,50.062763,19.923124,0.0,2025-04-30 00:47:19,stop_1626_311102,18,1,2025-04-30,3,24:47:00
2,594,block_524_trip_3_service_1,NaN,50.100731,20.013161,0.0,2025-04-30 00:47:26,stop_273_37802,1,1,2025-04-30,3,25:27:00


In [12]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests
import json

# Wczytanie danych GTFS-RT
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://gtfs.ztp.krakow.pl/TripUpdates_A.pb')
feed.ParseFromString(response.content)

entity_dict = MessageToDict(feed.entity[0], preserving_proto_field_name=True)

# Ładne wyświetlenie JSON
print(json.dumps(entity_dict, indent=2))

# Lista na dane
vehicles = []

# Przetwarzanie danych
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle

        vehicles.append({
            'trip_id': vehicle.trip_update.id,
            ''
        })

# Konwersja do DataFrame
df = pd.DataFrame(vehicles)

# Zapis do CSV (opcjonalnie)
df.to_csv('vehicle_positions.csv', index=False)

# Podgląd
print(df.head())

{
  "id": "block_974_trip_17",
  "trip_update": {
    "trip": {
      "trip_id": "block_974_trip_17_service_3"
    },
    "stop_time_update": [
      {
        "departure": {
          "time": "1745777400"
        },
        "stop_id": "stop_36_5401"
      },
      {
        "arrival": {
          "time": "1745777520"
        },
        "departure": {
          "time": "1745777520"
        },
        "stop_id": "stop_580_82202"
      },
      {
        "arrival": {
          "time": "1745777640"
        },
        "departure": {
          "time": "1745777640"
        },
        "stop_id": "stop_7_1203"
      },
      {
        "arrival": {
          "time": "1745777700"
        },
        "departure": {
          "time": "1745777700"
        },
        "stop_id": "stop_2309_345907"
      },
      {
        "arrival": {
          "time": "1745777760"
        },
        "departure": {
          "time": "1745777760"
        },
        "stop_id": "stop_105_13701"
      },
      {
        "